# RSI_Hedge_v1.4

## Setup

### Packages

In [1]:
from utils.packages import *

### Inputs and Parameters

#### Read Yaml files

In [2]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currs = yaml.load(temp_file)        

#### Yaml Variables

In [3]:
data = {}
data['instrument'] = "EUR_USD"
data['pip_size'] = currs['currs'][data['instrument']]['pip_size']

access_token = config['oanda_demo_hedge']['token']
accountID = config['oanda_demo_hedge']['account_id']
params = {'instruments': data['instrument']}

In [4]:
api = API(access_token = access_token)
request_stream_data = pricing.PricingStream(accountID=accountID, params=params)
response_stream = api.request(request_stream_data)

In [5]:
#--------------------------------------------------------------------------------------------------------------------------
def reset_data():
    global data
    #Declarations ------------------------------------------
    data['date_val'] = 0
    data['time_val'] = 0
    data['time_fraction'] = 0

    data['ask_price'] = 0
    data['bid_price'] = 0
    data['stop_price'] = 0
    data['tick_price'] = 0

    data['order_ask_price'] = 0
    data['order_bid_price'] = 0
    data['spread'] = 0
    
    data['RSI'] = 0
    data['rsi_status'] = None
    
    data['profit_val_long']      = 0
    data['max_profit_long']      = 0
    data['buffer_profit_long']   = 0
    
    data['profit_val_short']      = 0
    data['max_profit_short']      = 0
    data['buffer_profit_short']   = 0    

    data['create_order'] = False
    data['current_open_order'] = False
    data['positions_info'] = None
    data['long_positions'] = 0
    data['short_positions'] = 0

    data['order_response'] = 0
    data['close_response'] = 0

    data['tick_list'] = collections.deque([])
    data['UpPrice'] = collections.deque([])
    data['DownPrice'] = collections.deque([])
    data['AvgGain'] = collections.deque([])
    data['AvgLoss'] = collections.deque([])
    data['RS_List'] = collections.deque([])
    data['RSI_List'] = collections.deque([])


    #Parameters ------------------------------------------
    data['limit_iters'] = False
    data['iter_max'] = 180

    data['rsi_len'] = 14
    data['rsi_ob'] = 70
    data['rsi_os'] = 30
    
    data['max_rsi'] = 0
    data['min_rsi'] = 100

    data['order_num'] = 1

    data['stop_loss'] = 20
    data['stop_loss_pip'] = data['stop_loss'] * data['pip_size']

    data['take_profit_pip'] = 0.00005
    data['start_hedge_pip'] = 0.0002
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get timestamp of the price and segregate it
def get_date_time(resp):
    global data
    time_stamp = resp['time']
    data['date_val'], full_time = time_stamp.split(sep = 'T')
    data['time_val'], data['time_fraction'] = full_time.split(sep = '.')
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get bid and ask prices
def get_prices(resp):    
    global data
    data['bid_price'] = float(resp['bids'][0]['price'])    
    data['ask_price'] = float(resp['asks'][0]['price'])
    data['spread'] = data['ask_price'] - data['bid_price']
    data['tick_price'] = (data['ask_price'] + data['bid_price']) / 2
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def before_rsi_len():
    global data
    data['tick_list'].append(data['tick_price'])

    if len(data['tick_list']) == 1:
        data['UpPrice'].append(0)
        data['DownPrice'].append(0)
        data['AvgGain'].append(0)
        data['AvgLoss'].append(0)
        data['RS_List'].append(0)
        data['RSI_List'].append(0)
    elif len(data['tick_list']) > 1:
        old_price = data['tick_list'][len(data['tick_list'])-2]
        new_price = data['tick_price']
        if new_price > old_price:
            data['UpPrice'].append(new_price - old_price)
            data['DownPrice'].append(0)
            data['AvgGain'].append(np.mean(data['UpPrice']))
            data['AvgLoss'].append(np.mean(data['DownPrice']))
            data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
            data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
            data['RSI_List'].append(data['RSI'])            
        elif old_price > new_price:
            data['UpPrice'].append(0)
            data['DownPrice'].append(old_price - new_price)
            data['AvgGain'].append(np.mean(data['UpPrice']))
            data['AvgLoss'].append(np.mean(data['DownPrice']))
            data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
            data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
            data['RSI_List'].append(data['RSI'])            
        elif old_price == new_price:
            data['UpPrice'].append(0)
            data['DownPrice'].append(0)
            data['AvgGain'].append(np.mean(data['UpPrice']))
            data['AvgLoss'].append(np.mean(data['DownPrice']))
            data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
            data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
            data['RSI_List'].append(data['RSI'])            
    display.clear_output(wait = True)
    print(f"Building RSI Tick list : {len(data['tick_list'])}")
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def after_rsi_len():
    global data
    data['UpPrice'].popleft()
    data['DownPrice'].popleft()
    data['AvgGain'].popleft()
    data['AvgLoss'].popleft()
    data['RS_List'].popleft()
    data['RSI_List'].popleft()
    data['tick_list'].popleft()
    data['tick_list'].append(data['tick_price'])

    old_price = data['tick_list'][len(data['tick_list'])-2]
    new_price = data['tick_price']
    if new_price > old_price:
        data['UpPrice'].append(new_price - old_price)
        data['DownPrice'].append(0)
        data['AvgGain'].append(np.mean(data['UpPrice']))
        data['AvgLoss'].append(np.mean(data['DownPrice']))
        data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
        data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
        data['RSI_List'].append(data['RSI'])
    elif old_price > new_price:
        data['UpPrice'].append(0)
        data['DownPrice'].append(old_price - new_price)
        data['AvgGain'].append(np.mean(data['UpPrice']))
        data['AvgLoss'].append(np.mean(data['DownPrice']))
        data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
        data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
        data['RSI_List'].append(data['RSI'])
    elif old_price == new_price:
        data['UpPrice'].append(0)
        data['DownPrice'].append(0)
        data['AvgGain'].append(np.mean(data['UpPrice']))
        data['AvgLoss'].append(np.mean(data['DownPrice']))
        data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
        data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
        data['RSI_List'].append(data['RSI'])            

    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def check_for_open_orders():
    global data
    
    request_position_data = positions.OpenPositions(accountID=accountID)
    data['positions_info'] = api.request(request_position_data)

    if len(data['positions_info']['positions']) == 0:
        data['current_open_order'] = False

    elif len(data['positions_info']['positions']) == 1:
        data['long_positions'] = abs(int(data['positions_info']['positions'][0]['long']['units']))
        data['short_positions'] = abs(int(data['positions_info']['positions'][0]['short']['units']))

        if data['long_positions'] >= 1 and data['short_positions'] == 0:
            data['current_open_order'] = 'long'
        elif data['long_positions'] == 0 and data['short_positions'] >= 1:
            data['current_open_order'] = 'short'                        
        elif data['long_positions'] >= 1 and data['short_positions'] >= 1:
            data['current_open_order'] = 'both'            

    return()
#==========================================================================================================================    

#--------------------------------------------------------------------------------------------------------------------------
def check_rsi_status():
    global data
    if not data['current_open_order']:
        if data['RSI'] <= data['rsi_os']:
            data['rsi_status'] = 'OS'
        elif data['RSI'] >= data['rsi_ob']:
            data['rsi_status'] = 'OB'
        else:
            data['rsi_status'] = None
            data['create_order'] = None
    return()
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def rsi_peak_check():
    global data
    if not data['current_open_order']:
        if data['rsi_status'] == 'OB':
            data['max_rsi'] = max(data['max_rsi'], data['RSI'])
            if data['RSI'] < data['max_rsi']:
                data['create_order'] = 'short'
                data['max_rsi'] = 0
                
        if data['rsi_status'] == 'OS':
            data['min_rsi'] = min(data['min_rsi'], data['RSI'])
            if data['RSI'] > data['min_rsi']:
                data['create_order'] = 'long'
                data['min_rsi'] = 100                
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def make_order():
    global data
    if not data['current_open_order']:
        if data['create_order'] == 'long':            
            data['order_val'] = data['order_num'] * 1
            
            data['stop_price'] = data['ask_price'] - data['stop_loss_pip']
            stopLossOnFill = StopLossDetails(price=data['stop_price'])
                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)

            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['order_response'] = api.request(order_request_data)
            return()

        if data['create_order'] == 'short':
            data['order_val'] = data['order_num'] * -1
            
            data['stop_price'] = data['bid_price'] + data['stop_loss_pip']
            stopLossOnFill = StopLossDetails(price=data['stop_price'])

            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['order_response'] = api.request(order_request_data)
            return()
#==========================================================================================================================
          
    
#--------------------------------------------------------------------------------------------------------------------------
def take_profit():
    global data
    
    if data['current_open_order'] == 'both':
        data['order_ask_price'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['profit_val_long'] = data['bid_price'] - data['order_ask_price']
        data['order_bid_price'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['profit_val_short'] = data['order_bid_price'] - data['ask_price']
        data['profit_val_both'] =  data['profit_val_long'] + data['profit_val_short']
        
        if data['profit_val_both'] >= data['take_profit_pip']:
            data_long = {"longUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_long)
            data['close_response'] = api.request(order_close_data)
            
            data_short = {"shortUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_short)
            data['close_response'] = api.request(order_close_data)    
            
            check_for_open_orders()
            if not data['current_open_order']:
                reset_data()
    
    if data['current_open_order'] == 'long' or data['current_open_order'] == 'both':
        data['order_ask_price'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['profit_val_long']      = data['bid_price'] - data['order_ask_price']
        data['max_profit_long']      = max(data['max_profit_long'], data['profit_val_long'])
        data['buffer_profit_long']   = data['max_profit_long'] - data['take_profit_pip']
        data['buffer_profit_long']   = max(data['buffer_profit_long'], data['take_profit_pip'])
        if data['buffer_profit_long'] >= data['profit_val_long'] >= data['take_profit_pip']:
            data_long = {"longUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_long)
            data['close_response'] = api.request(order_close_data)
            
            check_for_open_orders()
            if not data['current_open_order']:
                reset_data()
                
    if data['current_open_order'] == 'short' or data['current_open_order'] == 'both':
        data['order_bid_price'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['profit_val_short']      = data['order_bid_price'] - data['ask_price']
        data['max_profit_short']      = max(data['max_profit_short'], data['profit_val_short'])
        data['buffer_profit_short']   = data['max_profit_short'] - data['take_profit_pip']
        data['buffer_profit_short']   = max(data['buffer_profit_short'], data['take_profit_pip'])
        if data['buffer_profit_short'] >= data['profit_val_short'] >= data['take_profit_pip']:
            data_short = {"shortUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_short)
            data['close_response'] = api.request(order_close_data)  
            
            check_for_open_orders()
            if not data['current_open_order']:
                reset_data()
    return()
#==========================================================================================================================
          
    
#--------------------------------------------------------------------------------------------------------------------------
def make_hedge():
    if data['current_open_order'] == 'long':
        data['order_ask_price'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['profit_val_long']      = data['bid_price'] - data['order_ask_price']
        if data['profit_val_long'] <= -data['start_hedge_pip']:
            data['order_val'] = data['order_num'] * -1
            data['stop_price'] = data['bid_price'] + data['stop_loss_pip']
            stopLossOnFill = StopLossDetails(price=data['stop_price'])
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            data['order_response'] = api.request(order_request_data)
            
    if data['current_open_order'] == 'short':
        data['order_bid_price'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['profit_val_short']      = data['order_bid_price'] - data['ask_price']
        if data['profit_val_short'] <= -data['start_hedge_pip']:
            data['order_val'] = data['order_num'] * 1            
            data['stop_price'] = data['ask_price'] - data['stop_loss_pip']
            stopLossOnFill = StopLossDetails(price=data['stop_price'])                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)            
            data['order_response'] = api.request(order_request_data)        
    return()
#==========================================================================================================================


In [6]:
reset_data()

for i, resp in enumerate(response_stream): 
    if data['limit_iters'] and i >= data['iter_max']:
        break
    if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        continue
    
    get_date_time(resp)
    get_prices(resp)
        
    if len(data['tick_list']) < data['rsi_len']:
        before_rsi_len()          
        continue
    elif len(data['tick_list']) == data['rsi_len']:
        after_rsi_len()        
        
    check_for_open_orders()    
    check_rsi_status()
    rsi_peak_check()
    
    make_order()
    check_for_open_orders()

    make_hedge()
    check_for_open_orders()      
    
    take_profit()
    check_for_open_orders()   
                
    print(pd.DataFrame.from_dict(data,orient='index'))
    display.clear_output(wait = True)

KeyboardInterrupt: 

In [10]:
print(json.dumps(data['close_response'], indent=2))

{
  "longOrderCreateTransaction": {
    "id": "1221",
    "accountID": "101-003-15069707-003",
    "userID": 15069707,
    "batchID": "1221",
    "requestID": "78790657290074954",
    "time": "2020-11-13T17:09:33.062848991Z",
    "type": "MARKET_ORDER",
    "instrument": "EUR_USD",
    "units": "-1",
    "timeInForce": "FOK",
    "positionFill": "REDUCE_ONLY",
    "reason": "POSITION_CLOSEOUT",
    "longPositionCloseout": {
      "instrument": "EUR_USD",
      "units": "ALL"
    }
  },
  "longOrderFillTransaction": {
    "id": "1222",
    "accountID": "101-003-15069707-003",
    "userID": 15069707,
    "batchID": "1221",
    "requestID": "78790657290074954",
    "time": "2020-11-13T17:09:33.062848991Z",
    "type": "ORDER_FILL",
    "orderID": "1221",
    "instrument": "EUR_USD",
    "units": "-1",
    "requestedUnits": "-1",
    "price": "1.18307",
    "pl": "0.0000",
    "quotePL": "0.00002",
    "financing": "0.0000",
    "baseFinancing": "0.00000000000000",
    "commission": "0.000

In [ ]:
output = pd.DataFrame()

In [ ]:
output = output.append(data, ignore_index=True)
output.head()

In [ ]:
output.to_csv('output.csv')

### streamline data and capture at orders 